### Deploying Machine Learning Models

After developing a ML model, we want to enable users to utilise the model without having to run any code. One means of doing this is through deploying the model to a server. Users can then make API calls and retrieve their requested data/predictions.

We will practice model deployment using Churn Predicition Project from weeks 3 and 4 of ML Zoomcamp (2023). Let us first run some code to train and evaluate the binary classification model.

In [5]:
# Importing libraries

# Data manipulation
import pandas as pd
import numpy as np

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Validation framework
from sklearn.model_selection import train_test_split

# Generate indices for K-Fold cross validation
from sklearn.model_selection import KFold

# Feature matrix formatter (dictionary vectoriser)
from sklearn.feature_extraction import DictVectorizer

# Logistic regression (sigmoid ver. linear regression)
from sklearn.linear_model import LogisticRegression

# ROC curves and AUC score
from sklearn.metrics import roc_auc_score

In [6]:
# Importing the data
df = pd.read_csv(r'..\Week 3\telco-customer-churn\WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Cleaning column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Storing categorical variables
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

# Cleaning contents of categorical series
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

# Parsing totalcharges as numeric (falsely parsed as an object)
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

# Converting .churn yes/no outcomes to binary (1/0)
df.churn = (df.churn == 'yes').astype(int)

In [7]:
# Establishing validation framework using sklearn
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [8]:
# Numerical series
numerical = ['tenure', 'monthlycharges', 'totalcharges']

# Categorical series
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [9]:
# Model for training classifier model
def train(df_train, y_train, C=1.0):

    # Initialising sklearn dictionary vectoriser (sparse matrix == False)
    dv = DictVectorizer(sparse=False)

    # Creating dictionary of training features
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    # Transforming dictionary to formatted feature matrix using DictVectoriser
    X_train = dv.fit_transform(dicts)

    # Initialising logistic regression model using sklearn
    model = LogisticRegression(C=C, max_iter=1000)

    # Training model on train split
    model.fit(X_train, y_train)

    return dv, model

In [10]:
# Function for making generating model predictions
def predict(df, dv, model):
    
    # Creating dictionary of training features
    dicts = df[categorical + numerical].to_dict(orient='records')
    
    # Transforming dictionary to formatted feature matrix using DictVectoriser
    X = dv.transform(dicts)

    # Obtaining probabilistic predictions for validation split
    y_pred = model.predict_proba(X)[:,1]

    return y_pred

In [11]:
# Model parameters
C = 1.0
n_splits = 5

In [12]:
# Generating indices for n_splits shuffled splits
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

# Empty array to store scores
scores = []

# Indices for train and validation parts
for train_idx, val_idx in kfold.split(df_full_train):

    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.841 +- 0.009


In [14]:
# Training the final model and validating on test split
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(df_test.churn.values, y_pred)
auc

0.8572386167896259

### Saving the Model

Our model currently lives inside a notebook; therefore, we cannot deploy the model to a webservice. Instead we need to extract (save) the model (python code) first so that we can generate a webservice using python frameworks, like Flask, Django, etc.

In [15]:
import pickle

In [18]:
# Specifying output name
output_file = f'model_C={C}.bin'

In [25]:
# Opening the file to edit (write bytes)
f_out = open(output_file, 'wb')

# Saving the trained model in output file
pickle.dump((dv, model), f_out)

# Closing the file - prevents accidently writing to file
f_out.close()

In [24]:
# Alternate, auto-closing method
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

### Loading the Model

Having saved the model using pickle, we want to test whether the file was written correctly. By restarting the kernel here, one can run the code below, reading the output file and verifying whether the model was successfully extracted or not.

In [1]:
import pickle

In [6]:
# Specifying file to read
input_file = 'model_C=1.0.bin'

# Reading input file
with open(input_file, 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

# Testing whether model saved correctly or not
model

### Testing Model on Dummy Customer

Having loaded the model, let us test the model's functionality on a dummy customer.

In [41]:
# Dummy customer profile
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [43]:
# Formatting feature matrix using extracted dictionary vectorizer
X = dv.transform([customer])

# Using model for prediction
customer_proba = model.predict_proba(X)[0,1]
print('Probability that customer will churn: {0:.1f}%'.format(customer_proba*100))

Probability that customer will churn: 63.6%


### Exporting Python Code

To serve the model to users we must first extract the python code from this notebook. This results in .py file(s) which we can then serve using python frameworks, like Flask. Converting the notebook (.ipynb) to a Python script (.py) is not difficult, but some modifications can be made to make the script more accessible/easy-to-understand.

### Serving the Model Using Flask

Having a Python script for training the model we can enable users to make requests by creating a prediction script which we then serve using Flask. Flask converts Python code into a GET/PULL/... webservice. In our case, the user will want to input information and be returned a prediction of whether or not a user will churn; hence, the PULL method is most appropriate.

Serving with Flask is not recommended outside of development; instead, we can use Waitress, a Python WSGI server, which is a windows-equivalent of Gunicorn for Linux. 

### Environment and Dependency Management

Unique services live in seperate environments to each other. This can lead to version inconsistencies, which may cause conflicts down the line. Virtual environments seperate/isolate processeses preventing potential conflicts.

Examples of virtual environment solutions:
* venv
* conda
* pipenv
* poetry

For this project we will use pipenv. Using pipenv, we can initialise an environment and be particular about dependecy versions, etc. One can then use `pipenv shell` to run commands using the environment.

### Docker

Docker enables us to isolate the application from other services/processes running on our local computer. Services are isolated in *containers* with their respective dependencies, software versions, etc. Using Docker enables us to compartmentalise our project which is useful for managing processes and workflow. For example, we may choose to upload one service to the cloud and this is straightforward if our service lives in its own compartment.

#### 1. Write the Dockerfile

#### 2. Build the Docker image
    
- Remembering to use the same Python version as pipfile<br>

#### 3. Run the Docker image
    
- Remembering to expose container port to host machine


Having creating a Docker instance, we can run the service inside the Docker container and access the service using an external script.

### Cloud Deployment

After containing the churn prediction service in a Docker container, there is no longer a need to host the container on a local machine. Instead, one can upload the container to a cloud service and access the prediction service locally.